# ErgoSCF methods: Basics

## Table of Content <a name="TOC"></a>

1. [General setups](#setups)

2. [Run ErgoSCF calculations](#run_ergoscf) 

3. [Read ErgoSCF calculations data and do some math with it](#read_ergoscf) 

4. [Optional cleanup](#cleanup) 

### A. Learning objectives

- to setup and run electronic structure calculations with ErgoSCF package
- to parse the output of the ErgoSCF calculations and convert them to Python data
- to do math with the electronic structure calculations data obtained from ErgoSCF


### B. Use cases

- using ErgoSCF data with Libra


### C. Functions

- `libra_py`
  - `data_outs`
    - [`print_matrix`](#print_matrix-1)
  - `ERGO_methods`
    - [`get_mtx_matrices`](#get_mtx_matrices-1)
    - [`read_mo_restricted`](#read_mo_restricted-1)
    - [`read_mo_unrestricted`](#read_mo_unrestricted-1)
    - [`read_spectrum_restricted`](#read_spectrum_restricted-1)
    - [`read_spectrum_unrestricted`](#read_spectrum_unrestricted-1)
    

### D. Classes and class members

- `liblibra::liblinalg`
  - `CMATRIX`
    - [`real`](#real-1)
    - [`H`](#H-1)
    - [`show_matrix`](#show_matrix-1)


## 1. General setups
<a name="setups"></a> [Back to TOC](#TOC)

In [1]:
import os
import sys

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

import util.libutil as comn
from libra_py import ERGO_methods
from libra_py import units
from libra_py import data_read, data_outs


/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<float, std::all

## 2. Run ErgoSCF calculations
<a name="run_ergoscf"></a> [Back to TOC](#TOC)

In this tutorial, we are going to use the [ErgoSCF](http://ergoscf.org/) package. 

> ### **<p style="color:red">Requirements:</p>**
>
>  - any version above 3.8 (at the point of writing this tutorial, this is the latest version)
>  - the "ergo" executable is added to the `$PATH` environment variable

In the next cell, we create an input script to run the ErgoSCF code.

Most of the input parameters are rather intuitive and i'm not going into many details. I only highlight some important points:

* coordinates are given in Angstrom 
* `scf.create_mtx_files_D = 1` - request to print out the density matrices in the AO basis. As a result, the following files are generated:
  - *D_matrix_alpha_\<num\>.mtx* and *D_matrix_beta_<num>.mtx*, where \<num\> is the index of the SCF iteration
* `scf.create_mtx_files_F = 1` - request to print out the Fock matrices in the AO basis. As a result, the following files are generated:
  - *F_matrix_alpha_\<num\>.mtx* and *F_matrix_beta_<num>.mtx*, where \<num\> is the index of the SCF iteration
* `scf.create_mtx_files_S = 1` - request to print out the AO overlap matrices. As a result, the following files are generated:
  - *S_matrix_alpha_\<num\>.mtx* and *S_matrix_beta_<num>.mtx*, where \<num\> is the index of the SCF iteration
* `scf.output_homo_and_lumo_eigenvectors = 1` - turn on the printing out of the computed MOs
* `scf.number_of_occupied_eigenvectors = 3` - request to print out 3 occupied orbitals, as a result, the following files will be produced:
  - *homo_coefficient_vec_alpha.txt* and *homo_coefficient_vec_beta.txt*  for **HOMO**
  - *occ_1_coefficient_vec_alpha.txt* and *occ_1_coefficient_vec_beta.txt*  for **HOMO-1**
  - *occ_2_coefficient_vec_alpha.txt* and *occ_2_coefficient_vec_beta.txt*  for **HOMO-2**  
* `scf.number_of_unoccupied_eigenvectors = 2` - request to print out 2 unoccupied orbitals, as a result, the following files will be produced:
  - *lumo_coefficient_vec_alpha.txt* and *lumo_coefficient_vec_beta.txt*  for **LUMO**
  - *unocc_1_coefficient_vec_alpha.txt* and *unocc_1_coefficient_vec_beta.txt*  for **LUMO+1**
* `scf.eigenvectors_method = "projection"` - needed to be there, for the orbital print outs to work

In addition to the files described above, the files that contain energies (in Ha units) of the requested occupied and unoccupied orbitals are also printed out in the following files:
  - *occupied_spectrum_alpha.txt* and *occupied_spectrum_beta.txt*
  - *unoccupied_spectrum_alpha.txt* and *unoccupied_spectrum_beta.txt*
  
The generation of the "alpha" and "beta" files is done for the unrestricted SCF calculations, which is requested via `scf.force_unrestricted = 1`

In [2]:
file_content = """ergo <<EOINPUT > /dev/null
spin_polarization = 0
charge = 0
molecule_inline
F    0  0  0.0
H    0  0  1.0
EOF
basis = "6-31G"
use_simple_starting_guess=1
tmpdir = "./tmp"
set_nthreads("detect")
scf.create_mtx_files_D = 1
scf.create_mtx_files_F = 1
scf.create_mtx_file_S = 1
scf.output_homo_and_lumo_eigenvectors = 1
scf.number_of_occupied_eigenvectors = 3
scf.number_of_unoccupied_eigenvectors = 2
scf.eigenvectors_method = "projection"
XC.sparse_mode = 1
scf.force_unrestricted = 1
scf.starting_guess_disturbance = 0.01

run "HF"

"""

f = open("run_ergoscf.sh", "w")
f.write(file_content)
f.close()

Once the file is generated, we can execute ErgoSCF. For this small problem, it can be run as simple as:

In [3]:
os.system("sh ./run_ergoscf.sh")

0

## 3. Read ErgoSCF calculations data and do some math with it
<a name="read_ergoscf"></a> [Back to TOC](#TOC)

The D_, F_, and S_ files in the .mtx format produced by ErgoSCF can be read into Libra's `CMATRIX` objects using the `get_mtx_matrices` function
<a name="get_mtx_matrices-1"></a>

In [4]:
help(ERGO_methods.get_mtx_matrices)

Help on function get_mtx_matrices in module libra_py.ERGO_methods:

get_mtx_matrices(filename, act_sp1=None, act_sp2=None)
    Get the matrices printed out by the DFTB+
    
    Args: 
        filename ( string ): the name of the file to read. In the MatrixMarket (mtx) format
        act_sp1 ( list of ints or None): the row active space to extract from the original files
            Indices here start from 0. If set to None - the number of AOs will be
            determined automatically from the file. [default: None]
        act_sp2 ( list of ints or None): the cols active space to extract from the original files
            Indices here start from 0. If set to None - the number of AOs will be
            determined automatically from the file. [default: None]
    
    Returns: 
        list of CMATRIX(N, M): X: where N = len(act_sp1) and M = len(act_sp2) 
            These are the corresponding property matrices (converted to the complex type)



<a name="show_matrix-1"></a><a name="print_matrix-1"></a><a name="real-1"></a>

In [5]:
S = ERGO_methods.get_mtx_matrices("S_matrix.mtx")
print("S"); S.show_matrix()
data_outs.print_matrix(S.real())

S
1.0  0.2443080145  0.1716329727  0.0  0.0  0.0  0.0  0.0  0.0  0.1285075482  0.08268807939  
0.2443080145  1.0  0.7635106529  0.0  0.0  0.0  0.0  0.0  0.0  0.5872040235  0.4422928659  
0.1716329727  0.7635106529  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.6968240601  0.7965108884  
0.0  0.0  0.0  1.0  0.0  0.0  0.4995168866  0.0  0.0  0.0  0.0  
0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.4995168866  0.0  0.4266534586  0.08480274693  
0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.4995168866  0.0  0.0  
0.0  0.0  0.0  0.4995168866  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
0.0  0.0  0.0  0.0  0.4995168866  0.0  0.0  1.0  0.0  0.5614608836  0.2960078781  
0.0  0.0  0.0  0.0  0.0  0.4995168866  0.0  0.0  1.0  0.0  0.0  
0.1285075482  0.5872040235  0.6968240601  0.0  0.4266534586  0.0  0.0  0.5614608836  0.0  1.0  0.6582920493  
0.08268807939  0.4422928659  0.7965108884  0.0  0.08480274693  0.0  0.0  0.2960078781  0.0  0.6582920493  1.0  


Here, I decided to print out only the real part of the constructed matrices (because they ARE real even though we store them in a complex format).

The orbital energies are read by the functions `read_spectrum_unrestricted` or `read_spectrum_restricted`
<a name="read_spectrum_unrestricted-1"></a>

In [6]:
help(ERGO_methods.read_spectrum_unrestricted)

Help on function read_spectrum_unrestricted in module libra_py.ERGO_methods:

read_spectrum_unrestricted()
    This function reads the eigenspectrum files
    produced by ErgoSCF in the case of spin-polarized
    (unrestricted) formulation.
    The names of the files are known, so the we do not need
    any arguments
    
    Note: the ordering of the orbitals in the ErgoSCF is:
    
    occupied_spectrum.txt       |    unoccupied_spectrum.txt
    ---------------------------------------------------------
    H                                       L
    H-1                                     L+1
    ...                                     ...
    
    So, the occupied orbitals order is reverted in the created array
    
    
    Args:  None
    
    Returns:
        (list, list): ( [e_a, e_b], [nocc, nvirt])
        
        * e_a ( list of floats ): energies of the alpha orbitals printed out
            this list contains both occupied and unoccupied orbitals
        * e_b ( list of 

In [7]:
[e_a, e_b], [nocc, nvirt] = ERGO_methods.read_spectrum_unrestricted() 
print(F"The number of occupied orbitals = {nocc}")
print(F"The number unof occupier orbitals = {nvirt}")
print(F"Energies of the alpha orbitals = {e_a}" )
print(F"Energies of the beta orbitals = {e_b}" )

The number of occupied orbitals = 3
The number unof occupier orbitals = 2
Energies of the alpha orbitals = [-0.884284, -0.676967, -0.676967, 0.278798, 1.36203]
Energies of the beta orbitals = [-0.884284, -0.676967, -0.676967, 0.278798, 1.36203]


To read the molecular orbitals, we use the `read_mo_unrestricted` or `read_mo_restricted` functions.
<a name="read_mo_unrestricted-1"></a>

In [8]:
help(ERGO_methods.read_mo_unrestricted)

Help on function read_mo_unrestricted in module libra_py.ERGO_methods:

read_mo_unrestricted(nocc, nvirt, act_space=None)
    This function reads the MO files
    produced by ErgoSCF in the case of spin-polarized
    (unrestricted) formulation.
    The names of the files are known, so the we do not need
    to specify that 
    
    Args:  
        nocc ( int ): the number of occupied orbitals in the pool of 
           the printed out MO files
        nvirt ( int ): the number of unoccupied (virtual) orbitals in the pool of 
           the printed out MO files
        act_space ( list of ints ): indices of the orbitals we are interested in
            The indexing convention is relative to HOMO, that is 0 is HOMO, -1 is HOMO-1,
            1 is LUMO, etc.
    
    Returns:
        ( list ): ( [mo_a, mo_b] )
        
        * mo_a ( CMATRIX(nao, nocc+nvirt) ): alpha orbitals 
        * mo_b ( CMATRIX(nao, nocc+nvirt) ): beta orbitals



In [9]:
[mo_a, mo_b] = ERGO_methods.read_mo_unrestricted(nocc, nvirt) 
data_outs.print_matrix(mo_a)

(-0.0938247+0j)  (-2.76384e-17+0j)  (2.98369e-18+0j)  (0.0734947+0j)  (-0.0413467+0j)  
(0.206471+0j)  (2.77759e-17+0j)  (-1.10379e-18+0j)  (-0.0731801+0j)  (0.117965+0j)  
(0.514182+0j)  (1.47245e-17+0j)  (1.44e-17+0j)  (-1.13243+0j)  (1.05287+0j)  
(-2.00866e-17+0j)  (0.670749+0j)  (5.6759e-10+0j)  (1.72781e-18+0j)  (-1.96493e-15+0j)  
(-0.574942+0j)  (1.96411e-17+0j)  (3.69367e-18+0j)  (-0.106776+0j)  (-0.638379+0j)  
(-6.26862e-18+0j)  (-5.6759e-10+0j)  (0.670749+0j)  (-3.2325e-18+0j)  (-6.05316e-15+0j)  
(8.82044e-18+0j)  (0.478801+0j)  (4.05163e-10+0j)  (-4.72598e-18+0j)  (-6.82723e-15+0j)  
(-0.216848+0j)  (1.8773e-17+0j)  (3.44825e-18+0j)  (-0.243165+0j)  (1.61215+0j)  
(-2.32594e-18+0j)  (-4.05163e-10+0j)  (0.478801+0j)  (-1.46093e-17+0j)  (-1.95543e-14+0j)  
(-0.35764+0j)  (-6.01955e-17+0j)  (-1.34439e-17+0j)  (-0.0146777+0j)  (-0.891614+0j)  
(-0.0406558+0j)  (-1.65669e-17+0j)  (-1.7499e-17+0j)  (1.74261+0j)  (-0.657494+0j)  


We may also decide to read only a subset of the orbitals printed out, by giving the `read_mo_unrestricted` function an additional parameter

In [10]:
# Only HOMO and LUMO
[mo_a, mo_b] = ERGO_methods.read_mo_unrestricted(nocc, nvirt, [0, 1]) 
data_outs.print_matrix(mo_a)

(2.98369e-18+0j)  (0.0734947+0j)  
(-1.10379e-18+0j)  (-0.0731801+0j)  
(1.44e-17+0j)  (-1.13243+0j)  
(5.6759e-10+0j)  (1.72781e-18+0j)  
(3.69367e-18+0j)  (-0.106776+0j)  
(0.670749+0j)  (-3.2325e-18+0j)  
(4.05163e-10+0j)  (-4.72598e-18+0j)  
(3.44825e-18+0j)  (-0.243165+0j)  
(0.478801+0j)  (-1.46093e-17+0j)  
(-1.34439e-17+0j)  (-0.0146777+0j)  
(-1.7499e-17+0j)  (1.74261+0j)  


In [11]:
# HOMO-1, HOMO and LUMO
[mo_a, mo_b] = ERGO_methods.read_mo_unrestricted(nocc, nvirt, [-1, 0, 1]) 
data_outs.print_matrix(mo_a)

(-2.76384e-17+0j)  (2.98369e-18+0j)  (0.0734947+0j)  
(2.77759e-17+0j)  (-1.10379e-18+0j)  (-0.0731801+0j)  
(1.47245e-17+0j)  (1.44e-17+0j)  (-1.13243+0j)  
(0.670749+0j)  (5.6759e-10+0j)  (1.72781e-18+0j)  
(1.96411e-17+0j)  (3.69367e-18+0j)  (-0.106776+0j)  
(-5.6759e-10+0j)  (0.670749+0j)  (-3.2325e-18+0j)  
(0.478801+0j)  (4.05163e-10+0j)  (-4.72598e-18+0j)  
(1.8773e-17+0j)  (3.44825e-18+0j)  (-0.243165+0j)  
(-4.05163e-10+0j)  (0.478801+0j)  (-1.46093e-17+0j)  
(-6.01955e-17+0j)  (-1.34439e-17+0j)  (-0.0146777+0j)  
(-1.65669e-17+0j)  (-1.7499e-17+0j)  (1.74261+0j)  


In [12]:
# Only HOMO-1 and LUMO+1
[mo_a, mo_b] = ERGO_methods.read_mo_unrestricted(nocc, nvirt, [-1, 2]) 
data_outs.print_matrix(mo_a)

(-2.76384e-17+0j)  (-0.0413467+0j)  
(2.77759e-17+0j)  (0.117965+0j)  
(1.47245e-17+0j)  (1.05287+0j)  
(0.670749+0j)  (-1.96493e-15+0j)  
(1.96411e-17+0j)  (-0.638379+0j)  
(-5.6759e-10+0j)  (-6.05316e-15+0j)  
(0.478801+0j)  (-6.82723e-15+0j)  
(1.8773e-17+0j)  (1.61215+0j)  
(-4.05163e-10+0j)  (-1.95543e-14+0j)  
(-6.01955e-17+0j)  (-0.891614+0j)  
(-1.65669e-17+0j)  (-0.657494+0j)  


We can verify the orthogonality of the MOs:

In [13]:
S = ERGO_methods.get_mtx_matrices("S_matrix.mtx")
[e_a, e_b], [nocc, nvirt] = ERGO_methods.read_spectrum_unrestricted()     
[mo_a, mo_b] = ERGO_methods.read_mo_unrestricted(nocc, nvirt) 

MO_ovlp_a = mo_a.H() * S * mo_a
MO_ovlp_b = mo_b.H() * S * mo_b

print("Alpha MO overlaps:")
data_outs.print_matrix(MO_ovlp_a.real())

print("Beta MO overlaps:")
data_outs.print_matrix(MO_ovlp_b.real())

Alpha MO overlaps:
0.9999992662854605  -1.0369497289083001e-17  -6.182642516871383e-18  9.190014209348174e-07  2.7119091646071736e-06  
-1.0369497289083008e-17  0.999999601700957  -1.0339757570035447e-25  -2.119411174995724e-17  -7.343219899387014e-15  
-6.182642516871385e-18  1.033975775007268e-25  0.999999601700957  -2.57093815265021e-17  -2.142392973432203e-14  
9.190014212123732e-07  -2.1194111749957228e-17  -2.57093815265021e-17  1.0000073863916374  -1.1484164739172797e-06  
2.7119091643018622e-06  -7.343219899387014e-15  -2.142392973432203e-14  -1.1484164737507463e-06  0.9999940625617334  
Beta MO overlaps:
0.9999992246529876  -2.373378355221236e-17  -3.976203774413897e-18  7.742751395589664e-07  2.6987061608613194e-06  
-2.3733783552212356e-17  0.999999601700957  1.8879681060733448e-32  4.4576322151325894e-17  2.9353324524013863e-15  
-3.976203774413884e-18  1.3454219514182278e-32  0.999999601700957  6.001679511762214e-17  2.2332941355000436e-15  
7.742751397983583e-07  4.457632

## Exercise 1:

Read the last Fock matrix and the eigenvectors and check that after the corresponding unitary transformation, one can get the diagonal matrix with the diagonal elements containing the eigenvalues yielded by the ErgoSCF.

All of the above calculations can be done for the restricted calculations as well. The usage of the corresponding  functions would be:
<a name="read_spectrum_restricted-1"></a><a name="read_mo_restricted-1"></a>

In [14]:
help(ERGO_methods.read_spectrum_restricted)
help(ERGO_methods.read_mo_restricted)

Help on function read_spectrum_restricted in module libra_py.ERGO_methods:

read_spectrum_restricted()
    This function reads the eigenspectrum files
    produced by ErgoSCF in the case of spin-unpolarized
    (restricted) formulation.
    The names of the files are known, so the we do not need
    any arguments
    
    Note: the ordering of the orbitals in the ErgoSCF is:
    
    occupied_spectrum.txt       |    unoccupied_spectrum.txt
    ---------------------------------------------------------
    H                                       L
    H-1                                     L+1
    ...                                     ...
    
    So, the occupied orbitals order is reverted in the created array
    
    Args:  None
    
    Returns:
        (list, list): ( [e_a], [nocc, nvirt])
        
        * e_a ( list of floats ): energies of the alpha orbitals printed out
            this list contains both occupied and unoccupied orbitals
        * nocc ( int ): the number of 

For instance:
<a name="H-1"></a>

In [15]:
# This code is nvere executed, but i just want it to be shown here as a pretty code
if 0:
    [e_a], [nocc, nvirt] = ERGO_methods.read_spectrum_restricted() 
    print(e_a, nocc, nvirt)

    [mo_a] = ERGO_methods.read_mo_restricted(nocc, nvirt) 
    print("mo_a.H() * S * mo_a = ");  (mo_a.H() * S * mo_a).show_matrix()

## 4. Optional cleanup
<a name="cleanup"></a> [Back to TOC](#TOC)

Change 0 to 1 if you want to run the instruction below - to remove all the files that were generated by this tutorial. Be sure not to run it in a different directory (in case you may have other files with the same extension)

In [16]:
if 0:
    os.system("rm D_matrix*.mtx F_matrix*.mtx S_matrix*.mtx \
              *_coefficient_*.txt *_spectrum_*.txt density.bin ergoscf.out \
              gabeditfile.gab run_ergoscf.sh")

## Exercise 2:

Setup a restricted SCF calculation with ErgoSCF on the same system. Read the produced eigenvectors and eigenvalues.  Repeat the examples of this tutorial but now with the restricted SCF data. Repeat the Exercise 1, but with the restricted SCF data.